In [1]:
import sys

sys.path.append("/home/td-user/rtrec")

# Movielens 1M

In [2]:
from rtrec.experiments.datasets import load_dataset

df = load_dataset(name='movielens_1m')

Using existing ratings.dat file.


In [3]:
df[['user', 'item', 'tstamp', 'rating']]

,user,item,tstamp,rating
0,1,1193,978300760,5
1,1,661,978302109,3
2,1,914,978301968,3
3,1,3408,978300275,4
4,1,2355,978824291,5
...,...,...,...,...
1000204,6040,1091,956716541,1
1000205,6040,1094,956704887,5
1000206,6040,562,956704746,5
1000207,6040,1096,956715648,4


In [4]:
# from rtrec.experiments.split import leave_one_last_item
# train_df, test_df = leave_one_last_item(df)

from rtrec.experiments.split import temporal_user_split
train_df, test_df = temporal_user_split(df)

#from rtrec.experiments.split import random_split
#train_df, test_df = random_split(df)

In [5]:
train_df

,user,item,rating,tstamp
0,6040,858,4,956703932
1,6040,593,5,956703954
2,6040,2384,4,956703954
3,6040,1961,4,956703977
4,6040,2019,5,956703977
...,...,...,...,...
797753,736,1278,4,1045711206
797754,736,3671,4,1045711217
797755,3840,1196,3,1046106127
797756,3840,1225,3,1046106162


In [6]:
from rtrec.recommender import Recommender
from rtrec.models import SLIM

#model = SLIM()
# nn_feature_selection is fast
model = SLIM(nn_feature_selection=50) 

recommender = Recommender(model)

In [7]:
recommender.bulk_fit(train_df)

Fitting SLIMElastic in parallel: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:02<00:00, 13.76it/s]

Fit completed in 4.16 seconds
Throughput: 191542.61 samples/sec


In [8]:
recommender.evaluate(test_df, recommend_size=10, filter_interacted=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:02<00:00, 22.92it/s]


{'precision': 0.13175496688742194,
 'recall': 0.06754951943786101,
 'f1': 0.07296676973664434,
 'ndcg': 0.14435388382113157,
 'hit_rate': 0.6355960264900662,
 'mrr': 0.2869143540418358,
 'map': 0.06889065511230252,
 'tp': 7958,
 'auc': 0.3382415576053828}

In [9]:
recommender.model._recommend(10, top_k=10, filter_interacted=True)

[1136, 595, 260, 1035, 2797, 1304, 2406, 1259, 34, 3421]

# test partial_fit

In [10]:
from rtrec.experiments.split import temporal_split

test_df1, test_df2 = temporal_split(test_df, test_frac=0.5)

In [11]:
from rtrec.recommender import Recommender
from rtrec.models import SLIM

model = SLIM(nn_feature_selection=50)
recommender = Recommender(model)

In [12]:
recommender.bulk_fit(train_df)

Fitting SLIMElastic in parallel: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:02<00:00, 13.71it/s]

Fit completed in 4.13 seconds
Throughput: 193001.08 samples/sec


### Note test_df1 is not fit

In [13]:
recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 22.15it/s]


{'precision': 0.14262763298821884,
 'recall': 0.067444457267907,
 'f1': 0.07371692361779726,
 'ndcg': 0.156132882438022,
 'hit_rate': 0.6522670474830418,
 'mrr': 0.3037711871610482,
 'map': 0.07673687338925943,
 'tp': 3995,
 'auc': 0.3510598397397755}

### Fit test_df1

In [14]:
recommender.fit(test_df1, update_interaction=True, parallel=True)

Fitting SLIMElastic in parallel: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:02<00:00, 14.28it/s]

Fit completed in 11.81 seconds
Throughput: 8574.39 samples/sec


In [15]:
recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 24.52it/s]


{'precision': 0.1460906818993221,
 'recall': 0.07070990690805568,
 'f1': 0.07688306714352752,
 'ndcg': 0.16107445337996862,
 'hit_rate': 0.6644055694394859,
 'mrr': 0.31611938480021357,
 'map': 0.07922980448079897,
 'tp': 4092,
 'auc': 0.3609835772938236}